# Problem

We consider the heat equation over a cross section of a heat sink
$$
\begin{align*}
  \partial_t T(\xi, t) & = c \Delta T(\xi, t), & \xi \in \Omega,\ t > 0, \\
  -c \nabla T(\xi, t) \cdot \vec{n} & = k_1 T(\xi, t), & \xi \in \Gamma_1,\ t > 0, \\
  -c \nabla T(\xi, t) \cdot \vec{n} & = k_2 u(t), & \xi \in \Gamma_2,\ t > 0, \\
  T(\xi, 0) & = 0, & \xi \in \Omega, \\
  y(t) & = \int_{\Gamma_2} T(\xi, t) \,\mathrm{d}\xi,
\end{align*}
$$
where $\Omega \subset \mathbb{R}^2$ is the domain, $\Gamma_2$ is the bottom boundary, $\Gamma_1 = \partial \Omega \setminus \Gamma_2$, $c = 100$, $k_1 = 0.1$, $k_2 = 1000$, $u(t)$ is the input, and $y(t)$ is the output.

# Imports and settings

In [ ]:
from pymor.basic import (set_defaults, InstationaryModel,
                         LTIModel, BTReductor,
                         ExpressionParameterFunctional)
import numpy as np
import dolfin as df
import mshr as ms
from matplotlib import pyplot as plt


set_defaults({
    'pymor.algorithms.lyapunov.solve_lyap_lrcf.default_sparse_solver_backend':
    'lradi',
    'pymor.algorithms.lradi.lyap_lrcf_solver_options.projection_shifts_init_seed':
    0
})

# FEniCS discretizer

In [ ]:
THREE_D = False  # True
WIDTH = 2
HEIGHT = 10
DEPTH = 5
FIN_OFFSET_RATIO = 0.1
FIN_COUNT = 10
FIN_VOL_FRACTION = 0.5
FIN_LENGTH = 6
RESOLUTION = 10


def _discretize():

    fin_height = HEIGHT * (1 - FIN_OFFSET_RATIO) / FIN_COUNT * FIN_VOL_FRACTION
    fin_offset = HEIGHT * FIN_OFFSET_RATIO
    fin_height_with_space = HEIGHT * (1 - FIN_OFFSET_RATIO) / FIN_COUNT

    if THREE_D:
        domain = ms.Box(df.Point(-WIDTH/2, 0., -DEPTH/2),
                        df.Point(WIDTH/2, HEIGHT, DEPTH/2))
    else:
        domain = ms.Rectangle(df.Point(-WIDTH/2, 0.),
                              df.Point(WIDTH/2, HEIGHT))

    for f in range(FIN_COUNT):
        ly = fin_offset + (f+1) * fin_height_with_space - fin_height
        if THREE_D:
            domain += ms.Box(df.Point(-FIN_LENGTH, ly, -DEPTH/2),
                             df.Point(FIN_LENGTH, ly+fin_height, DEPTH/2))
        else:
            domain += ms.Rectangle(df.Point(-FIN_LENGTH, ly),
                                   df.Point(FIN_LENGTH, ly+fin_height))

    mesh = ms.generate_mesh(domain, RESOLUTION)

    class Boundary(df.SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary

    class Bottom(df.SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and df.near(x[1], 0, 1e-14)

    bottom = Bottom()
    boundary = Boundary()
    boundary_markers = df.MeshFunction('size_t',
                                       mesh,
                                       mesh.geometric_dimension()-1,
                                       value=0)
    boundary.mark(boundary_markers, 1)
    bottom.mark(boundary_markers, 2)
    ds = df.Measure('ds', domain=mesh, subdomain_data=boundary_markers)

    V = df.FunctionSpace(mesh, 'P', 1)
    u = df.TrialFunction(V)
    v = df.TestFunction(V)

    MAT = df.assemble(df.Constant(100.)
                      * df.inner(df.grad(u), df.grad(v))
                      * df.dx
                      + df.Constant(0.1) * u * v * ds(1))
    F = df.assemble(df.Constant(1000.) * v * ds(2))
    MASS = df.assemble(u * v * df.dx)

    from pymor.operators.constructions import VectorOperator
    from pymor.bindings.fenics import (FenicsVectorSpace, FenicsMatrixOperator,
                                       FenicsVisualizer)
    from pymor.models.basic import InstationaryModel
    from pymor.algorithms.timestepping import ImplicitEulerTimeStepper

    # monkey patch apply_inverse_adjoint, assuming symmetry
    FenicsMatrixOperator.apply_inverse_adjoint = \
        FenicsMatrixOperator.apply_inverse

    # define parameter functionals (same as in
    # pymor.analyticalproblems.thermalblock)
    space = FenicsVectorSpace(V)
    op = FenicsMatrixOperator(MAT, V, V)
    rhs = VectorOperator(space.make_array([F]))
    mass = FenicsMatrixOperator(MASS, V, V, name='mass')

    # build model
    visualizer = FenicsVisualizer(space)
    time_stepper = ImplicitEulerTimeStepper(nt=100)
    d = InstationaryModel(10,  # final time
                                   space.zeros(),
                                   op,
                                   rhs,
                                   mass=mass,
                                   num_values=100,
                                   time_stepper=time_stepper,
                                   visualizer=visualizer)

    return d


def discretize(*args, **kwargs):
    from pymor.tools import mpi

    if mpi.parallel:
        from pymor.models.mpi import mpi_wrap_model
        return mpi_wrap_model(_discretize, use_with=True,
                              pickle_local_spaces=False)
    else:
        return _discretize()

# InstationaryModel

In [ ]:
fom = discretize()
fom = fom.with_(outputs={'output_functional': fom.rhs.H})

## Visualization

### Input function

In [ ]:
_input = ExpressionParameterFunctional('sin(pi*_t/3)**2', {'_t': ()})

### Trajectory

In [ ]:
traj_fom = fom.with_(rhs=fom.rhs * _input).solve()

### Storing for visualization in ParaView

In [ ]:
fom.visualize(traj_fom, filename='fig/fom.pvd')

### Snapshot at final time

In [ ]:
U = traj_fom[-1]
function = df.Function(U.space.V)
function.vector()[:] = U._list[0].impl
plt.figure()
p = df.plot(function, cmap='inferno')
plt.colorbar(p)
plt.axis('off')
plt.show()

# LTIModel

In [ ]:
lti = fom.to_lti().with_(solver_options={'lyap_lrcf': 'lradi'})

## Order, number of inputs, and number of outputs

In [ ]:
print(lti)

## Bode plot

In [ ]:
w = np.logspace(-2, 5, 20)

In [ ]:
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
plt.show()

## $\mathcal{H}_\infty$-norm
This is valid because the system is state-space symmetric ($E^T = E$, $A = A^T$, $C = B^T$).

In [ ]:
hinf_norm = lti.eval_tf(0)[0, 0]

# Balanced truncation

## Instatiating a reductor object

In [ ]:
reductor_bt = BTReductor(lti)

## $\mathcal{H}_\infty$-error bounds

In [ ]:
error_bounds = reductor_bt.error_bounds()[:20]

In [ ]:
hsv = lti.hsv()[:21]

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(range(1, len(error_bounds) + 1), error_bounds / hinf_norm, '.-')
ax.semilogy(range(1, len(hsv)), hsv[1:] / hinf_norm, '.-')
ax.set_xticks([1, 5, 10, 15, 20])
ax.grid()
ax.set_xlabel('Reduced order')
ax.set_ylabel(r'Relative $\mathcal{H}_\infty$-error')
plt.show()

## Reduction

In [ ]:
r = 10
rom_bt = reductor_bt.reduce(r)

In [ ]:
fig, ax = plt.subplots()
lti.mag_plot(w, ax=ax)
rom_bt.mag_plot(w, ax=ax)
plt.show()

## Error system

In [ ]:
err_bt = lti - rom_bt

In [ ]:
fig, ax = plt.subplots()
err_bt.mag_plot(w, ax=ax)
plt.show()

### Relative $\mathcal{H}_\infty$-error bounds

In [ ]:
hsv[r - 1] / hinf_norm, error_bounds[r - 1] / hinf_norm

### Relative $\mathcal{H}_2$-error

In [ ]:
err_bt.h2_norm() / lti.h2_norm()

## ROM visualization

### Form the ROM as a InstationaryModel to use the `solve` method

In [ ]:
rd = InstationaryModel(
    10,
    rom_bt.solution_space.zeros(),
    -rom_bt.A,
    rom_bt.B,
    mass=rom_bt.E,
    num_values=100,
    time_stepper=fom.time_stepper,
    visualizer=fom.visualizer,
)

### Trajectory of the ROM and its reconstruction in the high-order space

In [ ]:
traj_rom = rd.with_(rhs=rd.rhs * _input).solve()
traj_rom_rc = reductor_bt.V.lincomb(traj_rom.to_numpy())

### Reconstructed snapshot at final time

In [ ]:
U = traj_rom_rc[-1]
function = df.Function(U.space.V)
function.vector()[:] = U._list[0].impl
plt.figure()
p = df.plot(function, cmap='inferno')
plt.colorbar(p)
plt.axis('off')
plt.show()

## State error visualization

### Trajectory of the error system

In [ ]:
traj_err = traj_fom - traj_rom_rc

### Error at final time

In [ ]:
U = traj_err[-1]
function = df.Function(U.space.V)
function.vector()[:] = U._list[0].impl
vmax = U.sup_norm()[0]
plt.figure()
p = df.plot(function, cmap='coolwarm', vmin=-vmax, vmax=vmax)
plt.colorbar(p)
plt.axis('off')
plt.show()

## Output for the FOM, ROM and error

### Output (average temperature at the bottom boundary) for the FOM and ROM

In [ ]:
time_points = np.linspace(0, 10, 100)

C_norm = lti.C.array.l1_norm()[0]
output_fom = lti.C.apply(traj_fom).to_numpy()[:, 0] / C_norm
output_rom = rom_bt.C.apply(traj_rom).to_numpy()[:, 0] / C_norm

### Comparison of the FOM and ROM, and error

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)
ax0.plot(time_points, output_fom, label='FOM')
ax0.plot(time_points, output_rom, '--', label='ROM')
ax0.set_ylabel('temperature')
ax0.grid()
ax0.legend()

ax1.plot(time_points, output_fom - output_rom, 'tab:green', label='Error')
ax1.set_xlabel('time')
ax1.ticklabel_format(style='sci', axis='y', scilimits=(-3, -3), useMathText=True)
ax1.grid()
ax1.legend()
plt.show()